In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_tennis=pd.read_csv('/content/drive/My Drive/ML/lab3_2.csv')

In [8]:
attribute_names=list(df_tennis.columns)
attribute_names.remove('play')

In [9]:
def entropy_of_list(a_list):
    from collections import Counter
    cnt=Counter(x for x in a_list)
    num_instances=len(a_list)*1.0
    probs=[x/num_instances for x in cnt.values()]
    return entropy(probs)

In [10]:
def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2)for prob in probs])

In [11]:
total_entropy = entropy_of_list(df_tennis['play'])

In [12]:
def information_gain(df, split_attribute_name,target_attribute_name, trace=0):
  df_split=df.groupby(split_attribute_name)
  nobs=len(df.index)*1.0
  df_agg_ent=df_split.agg({target_attribute_name:[entropy_of_list,lambda x: len(x) / nobs]}) [target_attribute_name]
  df_agg_ent.columns=['Entropy','PropObservations']
  new_entropy =sum(df_agg_ent['Entropy'] * df_agg_ent['PropObservations'])
  old_entropy=entropy_of_list(df[target_attribute_name])
  return old_entropy - new_entropy

In [13]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
   
    from collections import Counter
    cut=Counter(x for x in df[target_attribute_name])

    if len(cut)==1:
        return next(iter(cut))
    elif df.empty or (not attribute_names):
        return default_class
    else:
      default_class=max(cut.keys())
      gainz = [information_gain(df,attr,target_attribute_name) for attr in attribute_names]
      index_of_max=gainz.index(max(gainz))
      best_attr = attribute_names[index_of_max]
      tree={best_attr:{}}
      remaining_attribute_names=[i for i in attribute_names if i!= best_attr]
      for attr_val, data_subset in df.groupby(best_attr):
        subtree=id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
        tree[best_attr][attr_val]=subtree
    return tree
   

In [ ]:
from pprint import pprint
tree = id3(df_tennis,'play',attribute_names)
print("\n\nThe Resultant Decision Tree is:\n")
pprint(tree)